<a href="https://colab.research.google.com/github/sudarshan-koirala/youtube-stuffs/blob/main/langchain/langchain_pandas_dataframe_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🐼📊 Pandas Dataframe Agent

## [Youtube Video covering this notebook](https://youtu.be/f3SWi14vFq4)

- This notebook shows how to use agents to interact with a pandas dataframe. It is mostly optimized for question answering.
- <font color="orange">Data analysis often requires synthesizing data across sources. Until recently, the CSV/Pandas agent could only inspect one dataframe at a time. Now, in langchain, the agent can access many dataframes at once.</font>

**NOTE: this agent calls the Python agent under the hood, which executes LLM generated Python code - this can be bad if the LLM generated Python code is harmful. Use cautiously.**

## ⚙️ Setup

In [ ]:
%%capture
!pip install langchain watermark openai

In [ ]:
%load_ext watermark
%watermark -a "Sudarshan Koirala" -vmp langchain,openai,pandas

Author: Sudarshan Koirala

Python implementation: CPython
Python version       : 3.10.11
IPython version      : 7.34.0

langchain: 0.0.186
openai   : 0.27.7
pandas   : 1.5.3

Compiler    : GCC 9.4.0
OS          : Linux
Release     : 5.15.107+
Machine     : x86_64
Processor   : x86_64
CPU cores   : 2
Architecture: 64bit



In [ ]:
import os
import openai
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# get your openai api key from https://platform.openai.com/account/api-keys 🔑
from getpass import getpass

OPENAI_API_KEY = getpass()
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
openai.api_key = os.getenv("OPENAI_API_KEY")

··········


In [ ]:
from langchain.agents import create_pandas_dataframe_agent
from langchain.llms import OpenAI

In [ ]:
import pandas as pd
url = "https://raw.githubusercontent.com/sudarshan-koirala/Logistic-Regression-for-Titanic-Dataset/master/Train_Titanic.csv"
df = pd.read_csv(url)
print(df.shape)
df.head()

(891, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## 📄 Single Dataframe Example
Agent can interact with single dataframe agent.

In [ ]:
agent = create_pandas_dataframe_agent(OpenAI(temperature=0), df, verbose=True)

In [ ]:
agent.run("how many rows are there?")



> Entering new AgentExecutor chain...
Thought: I need to count the number of rows
Action: python_repl_ast
Action Input: df.shape[0]
Observation: 891
Thought: I now know the final answer
Final Answer: There are 891 rows.

> Finished chain.


'There are 891 rows.'

In [ ]:
agent.run("how many people have more than 3 siblings")



> Entering new AgentExecutor chain...
Thought: I need to count the number of people with more than 3 siblings
Action: python_repl_ast
Action Input: df[df['SibSp'] > 3].shape[0]
Observation: 30
Thought: I now know the final answer
Final Answer: 30 people have more than 3 siblings.

> Finished chain.


'30 people have more than 3 siblings.'

## 📄📄 Multi DataFrame Example

Agent can also interact with multiple dataframes passed in as a list.

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [ ]:
df1 = df.copy()
df1["Age"] = df1["Age"].fillna(df1["Age"].mean())

In [ ]:
agent = create_pandas_dataframe_agent(OpenAI(temperature=0), [df, df1], verbose=True)
agent.run("how many rows in the age column are different?")



> Entering new AgentExecutor chain...
Thought: I need to compare the age columns in both dataframes
Action: python_repl_ast
Action Input: len(df1[df1['Age'] != df2['Age']])
Observation: 177
Thought: I now know the final answer
Final Answer: 177 rows in the age column are different.

> Finished chain.


'177 rows in the age column are different.'

In [ ]:
df2 = df1.copy()
df2["Age_multiplied"] = df1["Age"] * 2
df2.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age_multiplied
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,44.0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,76.0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,52.0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,70.0
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,70.0


In [ ]:
agent = create_pandas_dataframe_agent(OpenAI(temperature=0), [df, df1, df2], verbose=True)
agent.run("Are the number of columns same in all the dataframes?")



> Entering new AgentExecutor chain...
Thought: I need to compare the number of columns in each dataframe
Action: python_repl_ast
Action Input: len(df1.columns)
Observation: 12
Thought: I need to compare the number of columns in the other dataframes
Action: python_repl_ast
Action Input: len(df2.columns)
Observation: 12
Thought: I need to compare the number of columns in the last dataframe
Action: python_repl_ast
Action Input: len(df3.columns)
Observation: 13
Thought: I now know the final answer
Final Answer: No, the number of columns is not the same in all the dataframes.

> Finished chain.


'No, the number of columns is not the same in all the dataframes.'

**Hope you learned something new today ⛓️👨‍💻**